In [ ]:
import time
from datetime import datetime
import dask.dataframe as dd
from distributed import LocalCluster, Client
import spatialpandas as spd
from spatialpandas.geometry import (
    PointArray, MultiPointArray, LineArray,
    MultiLineArray, PolygonArray, MultiPolygonArray
)
from spatialpandas import GeoSeries, GeoDataFrame
%matplotlib inline

In [ ]:
cluster = LocalCluster(#silence_logs=logging.ERROR,
                       dashboard_address=':8790',
                       n_workers=4,
                       threads_per_worker=2,
                       memory_limit='3 GB')


client = Client(cluster)
client

In [4]:
cont_us_path = "/work/spd-scipy2020/data/contiguous_us_w_geohash.parquet"
ddf = dd.read_parquet(cont_us_path, columns=['latitude', 'longitude'])
display(ddf.head(2))
len(ddf)

,latitude,longitude
index,,
38605,24.396309,-75.881459
38630,24.396317,-109.907775


113944489

In [5]:
df = ddf.map_partitions(
    lambda df: GeoDataFrame(dict(
        position=PointArray(df[['longitude', 'latitude']]),
        **{col: df[col] for col in df.columns}
    ))
)

In [6]:
savepath = '/work/spd-scipy2020/data/us_cont_spatiallysorted.parquet'

In [7]:
t0 = time.time()
df.pack_partitions(npartitions=df.npartitions, shuffle='disk').to_parquet(savepath)
dt = time.time() - t0

In [8]:
with open(f'spatial_sort_time-{datetime.now()}.csv', 'w') as f:
    f.write(f'time_min,npartitions\n{dt/60},{df.npartitions}')
print('dt (s):', dt)

dt (s): 70.94613265991211


In [9]:
df = spd.io.read_parquet_dask(savepath)
display(df.head(2))
len(df)

,position,latitude,longitude
hilbert_distance,,,
25629,"Point([-124.443, 24.447])",24.447,-124.443
99850,"Point([-124.532, 24.704])",24.704,-124.532


113944489